In [ ]:
import requests
from bs4 import BeautifulSoup
from time import sleep
from tqdm.notebook import tqdm, trange
import pickle

In [ ]:
headers = {
    "Access-Control-Allow-Origin": "*",
    "Access-Control-Allow-Methods": "GET",
    "Access-Control-Allow-Headers": "Content-Type",
    "Access-Control-Max-Age": "3600",
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0",
}

In [ ]:
store_url = "https://locations.jackinthebox.com/us/"
loc_url = "https://locations.jackinthebox.com/"

In [ ]:
link_dic = {}

In [ ]:
page = requests.get(store_url, headers)
soup = BeautifulSoup(page.content, "html.parser")
loclist = soup.find(class_="Directory-listLinks")

In [ ]:
state_html = []
for li in loclist.find_all("li"):
    a = li.find(class_="Directory-listLink")
    link_dic[a["href"][3:5]] = []
    if len(a["href"]) > 30:
        link_dic[a["href"][:2]].append(loc_url+a["href"])
    else:
        state_html.append(loc_url+a["href"])

In [ ]:
print(len(state_html))
for i in state_html:
    print(i)

In [ ]:
print(link_dic)

In [ ]:
state_html.remove('https://locations.jackinthebox.com/us/in/indianapolis')
print(state_html)

In [ ]:
store_href = []
loop = tqdm(state_html)

for state in loop:
    loop.set_description(f"working on {state}")

    st_page = requests.get(state, headers)
    st_soup = BeautifulSoup(st_page.content, "html.parser")
    st_list = st_soup.find(class_="Directory-listLinks")

    for links in st_list.find_all("li"):
        g = links.find(class_="Directory-listLink")
        store_href.append(g["href"])
    sleep(0.5)

In [ ]:
print(store_href[0][3:])

In [ ]:
city_list = []
city_list.append('us/in/indianapolis')
for i in store_href:
    count = i.count('/')
    if count == 3:
        city_list.append(i[3:])
    elif count == 4:
        link_dic[i[6:8]].append(loc_url+i[3:])
print(city_list)

In [ ]:
print(link_dic)

In [ ]:
for city,n in zip(city_list,range(len(city_list))):
    city_list[n] = loc_url+city
print(city_list)

In [ ]:
final_list = []
loop1 = tqdm(city_list)

for city_store in loop1:
    loop1.set_description(f"working on {city_store}")

    city_page = requests.get(city_store, headers)
    city_soup = BeautifulSoup(city_page.content, "html.parser")
    cities = city_soup.find_all(class_="Teaser-titleLink")

    for links in cities:
        final_list.append(links["href"][2:])
    sleep(0.5)

In [ ]:
print(final_list)

In [ ]:
for i in final_list:
    link_dic[i[7:9]].append(loc_url+i[4:])

In [ ]:
print(link_dic)

In [ ]:
key_to_value_lengths = {k:len(v) for k, v in link_dic.items()}
print(sum([len(x) for x in link_dic.values()]))
print(key_to_value_lengths)

In [ ]:
print(link_dic["nm"])

In [ ]:
with open('/Users/benperry/Documents/Python/FastFood/LinkDicts/jackinthebox.pkl', 'wb') as f:
    pickle.dump(link_dic, f)